### First step

In the following step we are going to find the best joint (bid, pricing) using the implementation of Non Linear Programming optimization Algorithm provided by the Scipy Library

## Optimize with different bid

In [3]:
import numpy as np
import scipy.optimize as opt
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from customerManager import CustomerManager
customer_path = "./config/customer_classes.json"
cManager = CustomerManager(file_path = customer_path)


# optimize
b_p = (3.0,15.0)
b_b = (1.0 , 10.0)
bnds = (b_b, b_b, b_b, b_p)

methods_with_jac = ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-krylov','trust-exact']
methods = ['Nelder-Mead','Powell','CG','BFGS','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr']




### Over different starting points and methods

In [8]:
max_value = 0
best_method = 'none'
best_x = None

def objective( x):
    bids = x[:3]
    price = x[3]

    #Checks integrity
    if price < 3 or price > 15 : return 0.0 
    if any(b < 1 or b > 10 for b in bids): return 0.0


    res = cManager.revenue(bids,price,noise=False)

    return -res[0]

In [15]:
bids = np.linspace(1,10,10) 
prices = np.linspace(3,15,13)

X_tmp = []
X= []
n_starting_points = 50
#generate random starting point
for b1 in bids:
    for b2 in bids:
        for b3 in bids:
            for p in prices:
                X_tmp.append([b1,b2,b3,p])

for i in range(n_starting_points):
    X.append(X_tmp[np.random.choice(len(X_tmp))])

X= np.array(X)


In [16]:
for x0 in tqdm(X):
    for m in methods:
        solution = opt.minimize(objective, x0, method = m) 
        x = solution.x

        if max_value < -objective(x):
            max_value = -objective(x)
            best_method = m
            best_x = x
print("\n")
print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('bid 1 = ' + str(best_x[0]))
print('bid 2 = ' + str(best_x[1]))
print('bid 3 = ' + str(best_x[2]))
print('price = ' + str(best_x[3]))


100%|██████████| 50/50 [00:24<00:00,  2.06it/s]

Best method is TNC with a value of 1136.3047924873592
Solution
bid 1 = 3.9416202240278033
bid 2 = 3.0
bid 3 = 5.0
price = 6.999999987597111



In [17]:

print(-objective(best_x))

1136.3047924873592


### Optimize with one bid

In [22]:
max_value = 0
best_method = 'none'
best_x = None


def objective_2( x):
    bid = x[0]
    price = x[1]

    #Checks integrity
    if price < 3 or price > 15 : return 0.0 
    if bid < 1 or bid > 10: return 0.0
    bids = [bid,bid,bid]
    print(bids)
    res = cManager.revenue(bids,price,noise=False)
    return -res[0]

In [19]:
bids = np.linspace(1,10,10) 
prices = np.linspace(3,15,13)

X_tmp = []
X= []
n_starting_points = 50
#generate random starting point
for b in bids:
    for p in prices:
        X_tmp.append([b,p])

X= np.array(X_tmp)



In [20]:
for x0 in tqdm(X):
    for m in methods:
        solution = opt.minimize(objective, x0, method = m) 
        x = solution.x

        if max_value < -objective(x):
            max_value = -objective(x)
            best_method = m
            best_x = x

print("\n")
print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('bid = ' + str(best_x[0]))
print('price = ' + str(best_x[1]))

100%|██████████| 130/130 [00:44<00:00,  2.90it/s]

Best method is Powell with a value of 1136.3047913141731
Solution
bid = 3.94162024105148
price = 7.000000000018318



In [21]:

print(-objective(best_x))

1136.3047913141731
